## Загрузка библиотек

Здесь все как в [практике 2](https://colab.research.google.com/drive/1-0eknp0Qd4d2LxauhNwSZB4_Tb44UlJI?usp=share_link), но без подробностей.

In [1]:
import os
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings("ignore")

In [2]:
%%capture
!pip install pyspark

In [3]:
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator

sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
!mkdir /root/.kaggle
!touch /root/.kaggle/kaggle.json
!echo '{"username":"alexeyakulov","key":"13a773ea94aa434e23322891c65adc84"}' > /root/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
%%capture
!kaggle datasets download -d arashnic/book-recommendation-dataset
!unzip -q book-recommendation-dataset.zip

## Загрузка данных

Для колаборативной фильтрации нам понадобятся только айди пользователей, айди книжек и оценки, которые пользователи дали книжкам.

In [ ]:
ratings = pd.read_csv("Ratings.csv")
ratings.shape

(1149780, 3)

In [ ]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Приведем айди книжек в числовой вид.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(ratings["ISBN"].tolist())
ratings["ISBN"] = le.transform(ratings["ISBN"])
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,57188,0
1,276726,29750,5
2,276727,107392,0
3,276729,127253,3
4,276729,127287,6


In [ ]:
rating_non_zero = ratings[ratings["Book-Rating"] > 0]

In [ ]:
rating_non_zero.shape

(433671, 3)

## Рекоммендации с помощью pyspark

In [ ]:
ratings_spark = spark.createDataFrame(ratings)

In [ ]:
data_spark = (ratings_spark
    .select(
        'User-ID',
        'ISBN',
        'Book-Rating',
    )
).cache()

In [ ]:
(training, test) = data_spark.randomSplit([0.8, 0.2])

In [ ]:
als = ALS(maxIter=2, regParam=0.01,
          userCol="User-ID", itemCol="ISBN", ratingCol="Book-Rating",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Book-Rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

evaluator = RegressionEvaluator(metricName="mae", labelCol="Book-Rating",
                                predictionCol="prediction")

mae = evaluator.evaluate(predictions)
print("MAE = " + str(mae))

Root-mean-square error = 4.67051774701282
MAE = 2.7113144418628794


In [ ]:
predictions.show()

+-------+----+-----------+-------------+
|User-ID|ISBN|Book-Rating|   prediction|
+-------+----+-----------+-------------+
| 181817|   0|          6|          0.0|
|  11676|  38|          0|          0.0|
| 137397|  44|         10|  7.314229E-4|
| 269728|  47|          9|          0.0|
| 224450|  62|          0|  7.647016E-6|
| 179624|  62|          0| 1.5719885E-4|
| 236645|  62|          0|          0.0|
|  71285|  63|          7|-1.6762665E-4|
| 120548|  66|          0| 9.3942037E-4|
| 187520|  76|          0|          0.0|
| 158856| 102|          0|          0.0|
|  23902| 121|          9|  9.845776E-4|
|  93047| 137|          8| 1.6738032E-4|
|  10067| 153|          7|          0.0|
|  44925| 182|          0|          0.0|
| 244602| 200|          0|          0.0|
| 208410| 222|          9|          0.0|
|  11400| 231|          0|-0.0011613334|
|      8| 231|          5| -8.479137E-7|
|  11676| 231|          8|-3.5849004E-4|
+-------+----+-----------+-------------+
only showing top

In [ ]:
ratings_spark_0 = spark.createDataFrame(rating_non_zero)

In [ ]:
data_spark_0 = (ratings_spark_0
    .select(
        'User-ID',
        'ISBN',
        'Book-Rating',
    )
).cache()

In [ ]:
(training, test) = data_spark_0.randomSplit([0.8, 0.2])

In [ ]:
als = ALS(maxIter=2, regParam=0.01,
          userCol="User-ID", itemCol="ISBN", ratingCol="Book-Rating",
          coldStartStrategy="drop",
          implicitPrefs=True)
model = als.fit(training)

predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Book-Rating",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

evaluator = RegressionEvaluator(metricName="mae", labelCol="Book-Rating",
                                predictionCol="prediction")

mae = evaluator.evaluate(predictions)
print("MAE = " + str(mae))

Root-mean-square error = 7.882853091226896
MAE = 7.675016556962354


## Рекоммендации с помощью surprise

In [ ]:
%%capture
!pip install scikit-surprise

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

algo = SVD()
reader = Reader(rating_scale=(0, 11))
data = Dataset.load_from_df(ratings[["User-ID", "ISBN", "Book-Rating"]], reader)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.4971  3.4955  3.5031  3.4985  3.5070  3.5003  0.0042  
MAE (testset)     2.8123  2.8116  2.8196  2.8149  2.8193  2.8155  0.0034  
Fit time          28.34   27.94   28.90   28.33   28.91   28.48   0.37    
Test time         2.68    2.89    2.55    3.63    1.79    2.71    0.59    


{'test_rmse': array([3.4971485 , 3.49554321, 3.50307658, 3.4984977 , 3.50700765]),
 'test_mae': array([2.81226625, 2.81163849, 2.81958244, 2.81485007, 2.81929444]),
 'fit_time': (28.336252212524414,
  27.935471773147583,
  28.89670729637146,
  28.333229064941406,
  28.908143997192383),
 'test_time': (2.67864990234375,
  2.891946315765381,
  2.545215368270874,
  3.6319751739501953,
  1.7932815551757812)}

Видим, что обе реализации справляются с задачей рекоммендации на этом датасете, но не слишком хорошо. Рейтинг изменяется в диапазоне от 0 до 10, а обе модели ошибаются примерно на 2.5. Однако SVD работает чуть стабильней относительно ALS.

## Задание на лабораторную работу 1

В примерах были рассмотрены два варианта рекомендаций посредством колаборативной фильтрации. В качестве практического задания вам предлагается использовать содержание датасетов, чтобы предсказывать, какую оценку пользователь поставит книге.

Для этого представим задачу предсказания оценки как задачу регрессии, где целевой переменной будет признак Book-Rating.
Тогда план анализа данных будет следующим:
1. Предобработать данные: преобразовать все категориальные данные в числовые (для преобразования названий книжек в вектор попробуйте 3 подхода: [bag-of-words](https://scikit-learn.org/stable/modules/feature_extraction.html#common-vectorizer-usage), [tf-idf](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting), [word2vec](https://radimrehurek.com/gensim/models/word2vec.html), обработать пустые ячейки и неточности в столбцах.
2. Сделать нормировку данных.
3. Обучить и протестировать не менее трех моделей регрессии. Например, [линейную регрессию](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), [случайный лес](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) и [xgboost](https://xgboost.readthedocs.io/en/stable/python/python_intro.html#training). Можно также попробовать применить нейронную сеть. Например, по этому [туториалу](https://www.tensorflow.org/tutorials/keras/regression).
4. Оценить результаты (RMSE, MAE).

In [ ]:
# put your code here